Many fundamental features of authorial style can be implemented as simple functions over counts of words and sentences.  In this homework, you'll be implementing these functions to measure stylistic properties of a text.

For a detailed history of many of these techniques, see Grieve 2005, "[Quantiative Authorship Attribution: A History and Evaluation of Techniques](https://summit.sfu.ca/_flysystem/fedora/sfu_migrate/8840/etd1721.pdf)" (note this was a master's thesis).

Before working on this notebook, install seaborn:

```conda install seaborn```

This homework has 3 required questions (Q1-3) and one optional "check-plus" question (Q4) at the end.


First, let's read in some works of fiction by three different authors: [Henry James](https://en.wikipedia.org/wiki/Henry_James), [Jane Austen](https://en.wikipedia.org/wiki/Jane_Austen) and [Horatio Alger](https://en.wikipedia.org/wiki/Horatio_Alger).

In [ ]:
james=[]
for filename in ["../data/stylometry/208_daisy_miller_a_study.txt", "../data/stylometry/432_the_ambassadors.txt", "../data/stylometry/209_the_turn_of_the_screw.txt"]:
    with open(filename) as file:
        james.append(file.read())

austen=[]
for filename in ["../data/stylometry/158_emma.txt", "../data/stylometry/105_persuasion.txt", "../data/stylometry/1342_pride_and_prejudice.txt"]:
    with open(filename) as file:
        austen.append(file.read())

        
alger=[]
for filename in ["../data/stylometry/18581_adrift_in_new_york_tom_and_florence_braving_the_world.txt", "../data/stylometry/5348_ragged_dick_or_street_life_in_new_york_with_the_bootblacks.txt", "../data/stylometry/21632_fame.txt"]:
    with open(filename) as file:
        alger.append(file.read())


In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np

**Q1.** First, implement the average sentence length.  Write a function that takes a single text as input and returns the average length of sentences within it using `nltk.tokenize.word_tokenize` and `nltk.tokenize.sent_tokenize`.  The output should be a single real number.  

In [ ]:
def get_avg_sentence_length(text):
    # TODO

Test that function on texts by James, Austen and Alger (just execute this cell):

In [ ]:
for book in james:
    avg_sents=get_avg_sentence_length(book)
    print("James avg sentences: %.3f" % avg_sents)

for book in austen:
    avg_sents=get_avg_sentence_length(book)
    print("Austen avg sentences: %.3f" % avg_sents)

for book in alger:
    avg_sents=get_avg_sentence_length(book)
    print("Alger avg sentences: %.3f" % avg_sents)

**Q2**: Use the `nltk.word_tokenize` method to implement the type-token ratio (a measure of vocabulary diversity):

$$
\textrm{TTR} = {\textrm{number of distinct word types} \over \textrm{number of word tokens}}
$$


TTR is dependent on text length (intuitively, the longer a text is, the greater chance you have of a word type repeating), so this number is only comparable between documents of identical lengths.  Calculate this measure for the first 500 words of you document and report the results here. Exclude tokens that are exclusively punctuation from all counts, and calculate this measure over the lowercased version of that text.  (You can find puncutation with `string.punctuation`.)  Your `type_token_ratio` function should take a single input as argument (a text at least 500 tokens long) and output a single real number (the TTR for that text).


In [ ]:
def type_token_ratio(text, num_words=500):
    # TODO

Again, test that function on texts by James, Austen and Alger (just execute this cell):

In [ ]:
for book in james:
    ttr=type_token_ratio(book)
    print("James avg: %.3f" % ttr)

for book in austen:
    ttr=type_token_ratio(book)
    print("Austen avg: %.3f" % ttr)

for book in alger:
    ttr=type_token_ratio(book)
    print("Alger avg: %.3f" % ttr)
    

**Q3.** Finally, count up the token frequency of function words (these are determiners like "the", prepositions like "in" and conjunctions like "and" and "because").  Let's approximate "function words" by counting the token frequency of `nltk.corpus.stopwords`.  Here's that list:

In [ ]:
from nltk.corpus import stopwords
nltk_stopwords = set(stopwords.words('english'))
print(len(nltk_stopwords), nltk_stopwords)

Write a function `count_stopword_freqs` that takes a single text as input and returns a dictionary containing frequency of each of these terms within it -- e.g.:

{"wouldn": 0.00003, ..., "are": 0.0004}

Frequency is simply the count of that term in the text divided by the count of all tokens in the text.

In [ ]:
def count_stopwords_freqs(book):
    #TODO
            

Again, test that function on texts by James, Austen and Alger (just execute this cell):

In [ ]:
for book in james:
    freqs=count_stopwords_freqs(book)
    print("James 'because': %.5f" % freqs["because"])

for book in austen:
    freqs=count_stopwords_freqs(book)
    print("Austen 'because': %.5f" % freqs["because"])

for book in alger:
    freqs=count_stopwords_freqs(book)
    print("Alger 'because': %.5f" % freqs["because"])

The functions you wrote now give you a way represent a text as a vector of stylistic features.  Let's see how we can use that to compare the similarity of styles between the three authors above.

In [ ]:
def featurize_style(text):
    vec=[]
    vec.append(get_avg_sentence_length(text))
    vec.append(type_token_ratio(text))
    stopword_freq=count_stopwords_freqs(text)
    for stop in set(stopwords.words('english')):
        if stop in stopword_freq:
            vec.append(stopword_freq[stop])
        else:
            vec.append(0)
    return vec

In [ ]:
all_vecs=[]
for book in james:
    all_vecs.append(featurize_style(book))

for book in austen:
    all_vecs.append(featurize_style(book))

for book in alger:
    all_vecs.append(featurize_style(book))


We're going to compare the style vector for each book with each other, comparing them via cosine similarity. To give equal weight to each style dimension, we'll normalize each feature to its z score (with respect to the value of that feature across all observations).

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(all_vecs)
all_vecs=scaler.transform(all_vecs)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
sims=cosine_similarity(all_vecs, all_vecs)

Now let's visualize those similarites as a heatmap.

In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
names=["james1", "james2", "james3", "austen1", "austen2", "austen3", "alger1", "alger2", "alger3"]
df = pd.DataFrame(sims, columns=names)

In [ ]:
p1 = sns.heatmap(df, cmap="Greens", xticklabels=names, yticklabels=names)

Q4 **(check-plus)**: From the Grieve 2005, "[Quantiative Authorship Attribution: A History and Evaluation of Techniques](https://summit.sfu.ca/_flysystem/fedora/sfu_migrate/8840/etd1721.pdf)" paper referenced above, pick one other stylistic feature and implement it.  As with the other stylistic measures, run it on the 9 books above; does it differentiate those authors?